# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [4]:
data_link = 'https://raw.githubusercontent.com/fcgomezr/Udacity-Project-3/main/data%20sets/WA_Fn-UseC_-Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link, separator = ",")

In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data = complete_data.drop("customerID", axis=1)
complete_data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
0,7590-VHVEG,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,5575-GNVDE,0,0,0,34,1,0,1,0,1,...,1,0,0,0,0,0,1,1,0,0
2,3668-QPYBK,0,0,0,2,1,0,1,1,0,...,1,0,0,0,0,0,1,0,0,1
3,7795-CFOCW,0,0,0,45,0,0,1,0,1,...,1,0,0,1,0,0,0,1,0,0
4,9237-HQITU,0,0,0,2,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [7]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
3347,6923-AQONU,0,1,1,23,1,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0
6812,6563-VNPMN,1,0,0,45,1,0,1,1,0,...,0,1,0,1,0,0,0,0,0,0
5297,7647-GYYKX,0,1,1,54,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
5511,6586-MYGKD,0,1,0,70,1,1,1,0,0,...,1,0,0,0,1,0,0,0,1,0
5762,1421-HCERK,1,1,0,30,1,1,1,1,1,...,1,1,0,1,0,0,0,0,0,0


In [10]:
train_data.to_csv("traindata.csv")

In [11]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

Uploading an estimated of 12 files
Target already exists. Skipping upload for .amlignore
Target already exists. Skipping upload for .amlignore.amltmp
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for hyperparameter_tuning (1).ipynb
Target already exists. Skipping upload for hyperparameter_tuning.ipynb
Target already exists. Skipping upload for model_training.py
Target already exists. Skipping upload for README.md
Target already exists. Skipping upload for score.py
Target already exists. Skipping upload for train.py
Target already exists. Skipping upload for traindata.csv
Target already exists. Skipping upload for .ipynb_checkpoints/automl-checkpoint.ipynb
Target already exists. Skipping upload for __pycache__/train.cpython-36.pyc
Uploaded 0 files


In [12]:
temp = ftrain_data.drop_columns('Column1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_feaa7dba-4f2b-4bc6-93f6-8f0a2289bedf,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_feaa7dba-4f2b-4bc6-93f6-8f0a2289bedf,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  The training data had the following high cardinality features which were excluded for training. Please re

{'runId': 'AutoML_feaa7dba-4f2b-4bc6-93f6-8f0a2289bedf',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-26T23:14:54.384344Z',
 'endTimeUtc': '2021-09-26T23:46:14.124693Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-159386","workspace_name":"quick-starts-ws-159386","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Package:azureml-automl-runtime, training version:1.34.0.post1, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.2, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.2, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.0.post1, c

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('logisticregression', LogisticRegression(C=1.2067926406393288, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('26', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=3.727593720314938, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l1'

In [17]:
print(best_automl)

Run(Experiment: capstone-project,
Id: AutoML_feaa7dba-4f2b-4bc6-93f6-8f0a2289bedf_67,
Type: azureml.scriptrun,
Status: Completed)


In [18]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [19]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-26 23:57:52+00:00 Creating Container Registry if not exists..
2021-09-27 00:07:53+00:00 Registering the environment.
2021-09-27 00:07:54+00:00 Use the existing image.
2021-09-27 00:07:54+00:00 Generating deployment configuration.
2021-09-27 00:07:55+00:00 Submitting deployment to compute..
2021-09-27 00:07:58+00:00 Checking the status of deployment customerservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://a2972e52-c736-4d42-ab56-e2b87aafd6a3.southcentralus.azurecontainer.io/score
http://a2972e52-c736-4d42-ab56-e2b87aafd6a3.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
import requests
import json

# URL for the web service
scoring_uri = 'http://a2972e52-c736-4d42-ab56-e2b87aafd6a3.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
                "gender_Male" :0,
                "SeniorCitizen":1,
                "Dependents" :0,
                "Partner" :0,
                "tenure":2,
                "PhoneService" :1,
                "MultipleLines" :1,
                "InternetService_Fiber optic" :1,
                "InternetService_No":0,
                "OnlineSecurity" :0,
                "OnlineBackup" :1,
                "DeviceProtection" :0,
                "TechSupport" :0,
                "StreamingTV" :0,
                "StreamingMovies" :1,
                "Contract_One year" : 0,
                "Contract_Two year":0,
                "PaperlessBilling" :1,
                "PaymentMethod_Bank transfer (automatic)":0,
                "PaymentMethod_Credit card (automatic)":0,
                "PaymentMethod_Mailed check":0,
                "PaymentMethod_Electronic check" :1,
                "MonthlyCharges" : 55.7,
                "TotalCharges" : 239.8 }
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  [1]


In [22]:
# Web Service Logs
print(service.get_logs())

2021-09-27T00:12:53,971579300+00:00 - iot-server/run 
2021-09-27T00:12:53,981678000+00:00 - nginx/run 
2021-09-27T00:12:53,999731900+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-27T00:12:53,998866800+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_732632de80c129004c31329a710852e7/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-27T00:12:54,310459000+00:00 - iot-server/finish 1 0
2021-09-27T00:12:54,317058100+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (70)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 98
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-27 00:12:59,988 | root | INFO | Starting up app insights client
logging socket was 

In [26]:
# Delete the service
service.delete()